In [1]:
from dotenv import load_dotenv
import pprint

from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END

from nodes import run_agent_reasoning, tool_node

AGENT_REASON="agent_reasoning" 
ACT="act"
LAST=-1
load_dotenv()

def should_continue(state: MessagesState) -> str:
    """
    Determine whether to continue or end the agent loop.
    """
    last_message = state["messages"][LAST]
    if not last_message.tool_calls:
        return END
        
    return ACT
    
    

flow = StateGraph(MessagesState)
flow.add_node(AGENT_REASON, run_agent_reasoning)
flow.set_entry_point(AGENT_REASON)
flow.add_node(ACT, tool_node)
flow.add_edge(START, AGENT_REASON)

flow.add_conditional_edges(AGENT_REASON, should_continue , {ACT: ACT, END: END})
flow.add_edge(ACT, AGENT_REASON)

app = flow.compile()

In [3]:
initial_state = MessagesState(messages=[HumanMessage(content="What is the weather in Chennai and  Triple it.")] )
response = app.invoke(initial_state)

In [4]:
from pprint import pprint
pprint(response['messages'])

[HumanMessage(content='What is the weather in Chennai and  Triple it.', additional_kwargs={}, response_metadata={}, id='ee1f5d50-c5a6-46fa-ab0e-f36136269c54'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 1324, 'total_tokens': 1343, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1280}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CcVPVvmV8B53oJUKjnjSIpYcjhkBu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--9a25513b-7735-4ae4-9031-0caccdffdfcd-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current weather in Chennai'}, 'id': 'call_QjwLlpFm9J2WoYFUkc5ri5Sw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 